In [1]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime
import time

import os
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


In [2]:
now = datetime.now()
path = adress +'\\'+ now.strftime('%Y-%m-%d')
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\2023-09-19 already exists


# VNA init

In [3]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna


def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_ask_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_set_meas_points(points): 
    return int(vna.write('SENS1:SWE:POIN {}'.format(str(points))))  # should be 201


def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw(delay = 0):
    vna_on()    
    if delay !=0:
        vna.write("INIT1:IMM")
        # start measurement
        time.sleep(delay) # this part is important becouse the measurement nedds to finnish before readout process
        vna.write("CALC1:FORM MLOG")                        # format of data
        data_str = vna.query("CALC1:DATA? SDAT")            # readout
    else: 
        vna.write("INIT1:IMM")
        # start measurement
        time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
        vna.write("CALC1:FORM MLOG")                        # format of data
        data_str = vna.query("CALC1:DATA? SDAT")            # readout
        
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

def fft_f(bandwidth=10, nop=201):
    f = np.linspace(-bandwidth/2, bandwidth/2, nop)
    return f

def fft_A(A):
    fft_A = abs(fftshift(fft(A))**2)
    return fft_A

In [4]:
vna = vna_setup_CW() # initializing or checking for connection

Rohde-Schwarz,ZNB20-2Port,1311601062102129,3.45



In [73]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')


vna.write("INIT1:IMM")
vna_set_meas_points(401)
data_str = vna.query("CALC1:DATA? SDAT")            # readout
data = np.array(data_str.rstrip().split(",")).astype("float64")
s = data[0::2] + 1j * data[1::2]
ax[0].plot(abs(s))
ax[1].plot(angle(s))
time.sleep(10)

data_str = vna.query("CALC1:DATA? SDAT")            # readout
data = np.array(data_str.rstrip().split(",")).astype("float64")
s = data[0::2] + 1j * data[1::2]
ax[0].plot(abs(s))
ax[1].plot(angle(s))
time.sleep(10)

data_str = vna.query("CALC1:DATA? SDAT")            # readout
data = np.array(data_str.rstrip().split(",")).astype("float64")
s = data[0::2] + 1j * data[1::2]
ax[0].plot(abs(s))
ax[1].plot(angle(s))

In [64]:
vna_set_cw_freq(5.78195e9)
vna_set_power(-10)
vna_set_band(10)
vna_set_meas_points(201)

delay = 1

mag1, pha1 = vna_meas_cw()
time.sleep(delay)
mag2, pha2 = vna_meas_cw(delay)

In [34]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].loglog(fft_f(nop=40), fft_A(mag1))
ax[1].loglog(fft_f(nop=402), fft_A(pha1))

ax[0].loglog(fft_f(nop=402), fft_A(mag2))
ax[1].loglog(fft_f(nop=402), fft_A(pha2))

ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')

Text(0, 0.5, 'Phase')

In [55]:
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
end_time_1 = 41.1246
delta = end_time_1/201
time_1 = np.linspace(0, end_time_1, 201)
time_2 = np.linspace(end_time_1, 2*end_time_1, 201)+delta
time_tot = np.append(time_1, time_2)


ax[0].loglog(fft_A(time_1), fft_A(mag1[:201]))
ax[1].loglog(fft_A(time_1), fft_A(pha1[:201]))


ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')

Text(0, 0.5, 'Phase')

In [57]:
vna_set_meas_points(201)
vna.query('SENS1:SWE:TIME?')

'41.1246\n'

In [59]:
vna_set_meas_points(402)
vna.query('SENS1:SWE:TIME?')

'41.1246\n'